In [ ]:
"""
Fetch photometric data for ZTF candidates and save as CSV files.

This script reads ZTF candidate coordinates from a CSV file, queries the IRSA API
for photometry data, and stores the results in individual JSON files.
"""



import os
import requests

import pandas as pd
from tqdm import tqdm

# Constants
ZTF_CANDIDATES_FILE = "/media3/CRP8/TDE/data/preprocessed_alerts/extragalactic_find_DR_photometry.csv"
OUTPUT_SAVE_PATH = "/media3/rupesh/crp8/data/DR_photometry_full/output"
SEARCH_RADIUS_ARCSEC = 1.2
SEARCH_RADIUS_DEG = SEARCH_RADIUS_ARCSEC / 3600

API_BASE_URL = "http://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves"


def load_candidates(file_path):
    """Load ZTF candidates from a CSV file.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        pd.DataFrame: DataFrame containing ZTF candidate data.
    """
    return pd.read_csv(file_path)


def fetch_ztf_data_by_coordinates(ra, dec, radius_deg, output_format='CSV'):
    """Fetch photometry data from the SNAD API.

    Args:
        ra (float): Right ascension of the object.
        dec (float): Declination of the object.
        radius_arcsec (float): Search radius in arcseconds.

    Returns:
        dict: JSON response containing photometry data.
    """
    # Convert arcsec to degrees

    # Base URL
    url = "http://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves"

    # Params to be passed
    params = {
        "POS": f"CIRCLE {ra} {dec} {radius_deg:.6f}",
        "FORMAT": output_format.upper()
    }

    # Perform GET request
    response = requests.get(url, params=params)

    if response.status_code == 200 and response.text.strip():
        return response.text
    else:
        return None


def save_csv(file_path, csv_content):
    """Save JSON content to a file.

    Args:
        file_path (str): Path to save the JSON file.
        csv_content (dict): JSON content to write.
    """
    with open(file_path, "w") as csv_file:
        csv_file.write(csv_content)


def process_candidates(candidates_df, output_path):
    """Process each candidate and save corresponding photometry data.

    Args:
        candidates_df (pd.DataFrame): DataFrame of ZTF candidates.
        output_path (str): Directory to save JSON files.
    """
    for _, row in tqdm(candidates_df.iterrows(), total=len(candidates_df)):
        ra = row["mean_ra"]
        dec = row["mean_dec"]
        object_id = row["objectId"]

        try:
            data = fetch_ztf_data_by_coordinates(ra, dec, SEARCH_RADIUS_DEG)
            output_file = os.path.join(output_path, f"{object_id}.csv")
            save_csv(output_file, data)
        except requests.RequestException as e:
            print(f"Failed to fetch data for {object_id}: {e}")


def main():
    candidates = load_candidates(ZTF_CANDIDATES_FILE)
    os.makedirs(OUTPUT_SAVE_PATH, exist_ok=True)
    process_candidates(candidates, OUTPUT_SAVE_PATH)


if __name__ == "__main__":
    main()
